<h1 id="tocheading">Table of Contents</h1>
<div id="toc"></div>

In [1]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

# Prerequisites
Please make sure the following Python distributions and packages were installed.

* [Anaconda](https://anaconda.org)
* [XGBoost](https://github.com/dmlc/xgboost)
* [seaborn](https://seaborn.pydata.org)

You'll also need to create the following sub-folders in your working folder:

* data
   
   To store all the data files downloaded from Lending Club
   To store processed data file

# Data Processing

* Fetch current listed loans from lending club website api, save as current_list.txt 

* Unify feature formats for current loans (i.e. change the name to lower case) and 2014 loans (i.e. remove '_')

* Find common features of current and 2014 loans, (92 common feature in total),  only keep common features shared by both current and 2014 loans
* save to loan_2014_processed.csv

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import json
import requests

# 1. Get Current Data from Lending Club API

current loan data that are available for investing right now is downloanded from Lending Club API. Current loan data is useful to find out which featues about loans are currently provided by Lending Club and can be used to compare the difference to historical data

The Apikey was copied from API interface and saved in apikeys.txt, 

In [4]:
# Use lending club website to download data
# api key is saved in apikeys.txt
api_key =open('apikeys.txt','r').read()

headers = {'Authorization':api_key}
url='https://api.lendingclub.com/api/investor/v1/loans/listing'

r=requests.get(url, headers=headers)

In [5]:
# loan data saved in r.json()
type(r.json())

dict

In [6]:
r.json()

{'asOfDate': '2018-05-12T17:38:33.414-07:00',
 'loans': [{'accNowDelinq': 0,
   'accOpenPast24Mths': 1,
   'acceptD': '2018-05-12T13:31:59.000-07:00',
   'addrState': 'IL',
   'addrZip': '604xx',
   'allUtil': 64.9,
   'annualInc': 30000.0,
   'annualIncJoint': 97000.0,
   'applicationType': 'JOINT',
   'avgCurBal': 21459,
   'bcOpenToBuy': 574,
   'bcUtil': 64.1,
   'chargeoffWithin12Mths': 0,
   'collections12MthsExMed': 0,
   'creditPullD': '2018-05-12T13:21:12.000-07:00',
   'delinq2Yrs': 0,
   'delinqAmnt': 0.0,
   'desc': None,
   'disbursementMethod': 'CASH',
   'dti': 12.2,
   'dtiJoint': 15.25,
   'earliestCrLine': '1991-06-11T17:00:00.000-07:00',
   'empLength': 0,
   'empTitle': 'Trimmer',
   'expD': '2018-06-11T14:00:00.000-07:00',
   'expDefaultRate': 3.95,
   'ficoRangeHigh': 744,
   'ficoRangeLow': 740,
   'fundedAmount': 2650.0,
   'grade': 'B',
   'homeOwnership': 'MORTGAGE',
   'housingPayment': 997.0,
   'iLUtil': 56.2,
   'id': 133312234,
   'ilsExpD': '2018-05-12T1

In [7]:
# save loan data in json as myData
data = r.json()
myData =data['loans']

## 1.1 save current loan data pulled from API into *current_list.txt *load current list into *df_current*


Loan data from Lending Club is in Json format and it should be saved into local txt file. 

In [8]:
# dump myData into current_list.txt 
with open('data/current_list.txt', 'w') as outfile:
    json.dump(myData, outfile)

In [9]:
# load current loan info into df_current from current_list.txt
df_current=pd.read_json('data/current_list.txt')

In [10]:
# check df_current
df_current.head()

,accNowDelinq,accOpenPast24Mths,acceptD,addrState,addrZip,allUtil,annualInc,annualIncJoint,applicationType,avgCurBal,...,totCollAmt,totCurBal,totHiCredLim,totalAcc,totalBalExMort,totalBalIl,totalBcLimit,totalCuTl,totalIlHighCreditLimit,totalRevHiLim
0,0,1,2018-05-12T13:31:59.000-07:00,IL,604xx,64.9,30000,97000.0,JOINT,21459,...,0,107295,122353,11,6827,1695,1600,0,3017,7500
1,0,7,2018-05-12T12:52:47.000-07:00,CA,957xx,90.1,40000,NaN,INDIVIDUAL,4876,...,3174,43882,48729,15,43882,38922,7900,1,39829,8700
2,0,2,2018-05-09T16:11:02.000-07:00,FL,331xx,39.3,40000,NaN,INDIVIDUAL,1242,...,0,7452,18943,6,7452,284,5800,0,10243,8700
3,0,5,2018-05-12T13:22:41.000-07:00,CA,920xx,32.9,39000,NaN,INDIVIDUAL,493,...,0,1972,6000,6,1972,0,4500,1,0,6000
4,0,7,2018-05-10T18:26:21.000-07:00,AZ,852xx,51.7,40000,NaN,INDIVIDUAL,12899,...,0,270871,238550,37,244128,224976,22200,2,165150,38400


In [11]:
# check df_current
df_current.tail()

,accNowDelinq,accOpenPast24Mths,acceptD,addrState,addrZip,allUtil,annualInc,annualIncJoint,applicationType,avgCurBal,...,totCollAmt,totCurBal,totHiCredLim,totalAcc,totalBalExMort,totalBalIl,totalBcLimit,totalCuTl,totalIlHighCreditLimit,totalRevHiLim
14,0,1,2018-05-10T17:19:54.000-07:00,NJ,079xx,5.0,210000,227340.0,JOINT,11794,...,0,129738,231000,26,129738,0,62000,0,0,231000
15,0,1,2018-05-10T20:05:38.000-07:00,FL,336xx,61.9,94000,NaN,INDIVIDUAL,18486,...,0,203350,251750,22,64130,21669,56300,2,31350,72200
16,0,12,2018-05-10T20:50:44.000-07:00,AR,727xx,56.0,72000,NaN,INDIVIDUAL,6690,...,1318,127119,153486,24,29446,11311,20000,0,14975,36300
17,0,6,2018-05-07T20:48:59.000-07:00,TX,798xx,39.2,110000,275000.0,JOINT,42205,...,0,379844,413098,29,68254,65236,6000,1,84070,7700
18,0,5,2018-05-12T12:59:26.000-07:00,GA,309xx,49.3,50400,NaN,INDIVIDUAL,1277,...,0,14042,28495,17,14042,7391,6800,3,8095,20400


In [12]:
# check current data shape
df_current.shape

(19, 119)

note that lending club provides 119 features for current loans

# 2. Load historical data in 2014

In this project, we are only using loans with 36 months terms. 

For model training, we must focus on loans that have been completed, in which the final status of the loan, i.e. Paid or Defaulted, is known. Consequently, all the loans issued after 2015 needs to be eliminated because of their unmatured status. 

Additionally, because of the time sensitive nature of loan data, we plan to use the newest data available, which are the loans originated in 2014.

In [13]:
# historical loan data in loan_2014.csv
df_2014 = pd.read_csv('Data/loan_2014.csv', skiprows=0)

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (5,46) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
# check historical loan data
df_2014.head()

,numtl120dpd2m,totalbalexmort,inqfi,accopenpast24mths,avgcurbal,id,mosinoldrevtlop,openacc,numrevtlbalgt0,inqlast6mths,...,intrate,addrstate,zipcode,fundedamnt,verificationstatus,verificationstatusjoint,loanamnt,numacctsever120pd,issued,loanstatus
0,0.0,15030.0,NaN,7.0,9536.0,36805548,290.0,17.0,9.0,2.0,...,6.99%,CA,937xx,10400.0,Not Verified,NaN,10400.0,4.0,Dec-2014,Charged Off
1,0.0,149140.0,NaN,5.0,29828.0,38098114,244.0,6.0,4.0,0.0,...,12.39%,VA,235xx,15000.0,Source Verified,NaN,15000.0,0.0,Dec-2014,Fully Paid
2,0.0,38566.0,NaN,8.0,3214.0,37822187,265.0,12.0,7.0,0.0,...,13.66%,NJ,077xx,9600.0,Source Verified,NaN,9600.0,0.0,Dec-2014,Fully Paid
3,0.0,64426.0,NaN,6.0,5857.0,37662224,148.0,11.0,4.0,1.0,...,13.66%,AZ,850xx,7650.0,Source Verified,NaN,7650.0,0.0,Dec-2014,Charged Off
4,0.0,18007.0,NaN,2.0,32727.0,37612354,170.0,8.0,5.0,0.0,...,17.14%,CA,953xx,12800.0,Verified,NaN,12800.0,1.0,Dec-2014,Current


In [15]:
# check historical loan data
# note that there are two NAN rows at the end of the dataframe
df_2014.tail()

,numtl120dpd2m,totalbalexmort,inqfi,accopenpast24mths,avgcurbal,id,mosinoldrevtlop,openacc,numrevtlbalgt0,inqlast6mths,...,intrate,addrstate,zipcode,fundedamnt,verificationstatus,verificationstatusjoint,loanamnt,numacctsever120pd,issued,loanstatus
235626,0.0,50896.0,NaN,7.0,28172.0,9604874,130.0,21.0,3.0,1.0,...,7.90%,CA,913xx,2000.0,Verified,NaN,2000.0,0.0,Jan-2014,Fully Paid
235627,0.0,73598.0,NaN,8.0,4089.0,9584776,93.0,18.0,2.0,2.0,...,16.99%,OH,458xx,20700.0,Verified,NaN,20700.0,0.0,Jan-2014,Current
235628,0.0,57477.0,NaN,4.0,11495.0,9199665,166.0,6.0,2.0,0.0,...,19.20%,CA,950xx,10000.0,Verified,NaN,10000.0,0.0,Jan-2014,Fully Paid
235629,NaN,NaN,NaN,NaN,NaN,Total amount funded in policy code 1: 3503840175,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
235630,NaN,NaN,NaN,NaN,NaN,Total amount funded in policy code 2: 873652739,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# check the shape of the dataframe
df_2014.shape

(235631, 94)

In [17]:
# features existing in current data
print ('----------current-----------------')
for col in df_current.columns.values:
    print (col)
print ('----------current-----------------')

----------current-----------------
accNowDelinq
accOpenPast24Mths
acceptD
addrState
addrZip
allUtil
annualInc
annualIncJoint
applicationType
avgCurBal
bcOpenToBuy
bcUtil
chargeoffWithin12Mths
collections12MthsExMed
creditPullD
delinq2Yrs
delinqAmnt
desc
disbursementMethod
dti
dtiJoint
earliestCrLine
empLength
empTitle
expD
expDefaultRate
ficoRangeHigh
ficoRangeLow
fundedAmount
grade
homeOwnership
housingPayment
iLUtil
id
ilsExpD
initialListStatus
inqFi
inqLast12m
inqLast6Mths
installment
intRate
investorCount
isIncV
isIncVJoint
listD
loanAmount
maxBalBc
memberId
moSinOldIlAcct
moSinOldRevTlOp
moSinRcntRevTlOp
moSinRcntTl
mortAcc
mtgPayment
mthsSinceLastDelinq
mthsSinceLastMajorDerog
mthsSinceLastRecord
mthsSinceRcntIl
mthsSinceRecentBc
mthsSinceRecentBcDlq
mthsSinceRecentInq
mthsSinceRecentRevolDelinq
numAcctsEver120Ppd
numActvBcTl
numActvRevTl
numBcSats
numBcTl
numIlTl
numOpRevTl
numRevAccts
numRevTlBalGt0
numSats
numTl120dpd2m
numTl30dpd
numTl90gDpd24m
numTlOpPast12m
openAcc
openAcc6

In [18]:
# features existing in current data
print ('----------2014-----------------')
for col in df_2014.columns.values:
    print (col)
print ('----------2014-----------------')

----------2014-----------------
numtl120dpd2m
totalbalexmort
inqfi
accopenpast24mths
avgcurbal
id
mosinoldrevtlop
openacc
numrevtlbalgt0
inqlast6mths
totalilhighcreditlimit
revolutil
grade
mortacc
mthssincercntil
mthssincelastrecord
tothicredlim
memberid
mthssincerecentbc
numtl90gdpd24m
maxbalbc
numiltl
totalrevhilim
mthssincerecentbcdlq
ficorangelow
mthssincerecentrevoldelinq
initialliststatus
numbcsats
numsats
bcopentobuy
openil24m
applicationtype
delinqamnt
openrv24m
percentbcgt75
taxliens
installment
ficorangehigh
mosinoldilacct
subgrade
numactvrevtl
numtloppast12m
openrv12m
numactvbctl
chargeoffwithin12mths
numbctl
desc
allutil
ilutil
pubrec
bcutil
pcttlnvrdlq
totalcutl
mthssincelastmajorderog
accnowdelinq
totalbalil
totcollamt
mthssincerecentinq
mosinrcntrevtlop
openil12m
totalbclimit
dtijoint
homeownership
pubrecbankruptcies
numoprevtl
revolbal
annualinc
totalacc
mthssincelastdelinq
delinq2yrs
collections12mthsexmed
numtl30dpd
purpose
annualincjoint
mosinrcnttl
openacc6m
term
to

# 3. find commom features shared by current and historical data

There are only 95 features in the historical data in 2014, while there are 119 features in current data obtained in April, 2018. This is probably due to the change in the information gathering by Lending Club.

It is necessary to find out common features shared by both 2014 data and current data, since we are going to use 2014 historical data to predict loan default in current data. Features that are not shared by 2014 and current data are not useful for our model and need to be removed.

In [19]:
current_feature = list(df_current.columns.values)

## processing of feature name for current data and historical data

Additionally, it has been noticed that there are also some changes in feature names between 2014 and 2018, i.e. some features in 2014 loans may have different names in current loans. So it is necessary to unify the nomenclature of features for current loans and 2014 loans. 

One additional difference between 2014 loan features and current loan features comes from the format of the feature names. For example, some features in current loans are named following the Camel-Case guidance, while some features in 2014 are using underscore (“_”). Features names should be formatted using the same rule before usage. 

In [20]:
# convert current feature into lower case
for i in range (len(current_feature)):
    current_feature[i] = str.lower(str(current_feature[i]))

In [21]:
current_feature

['accnowdelinq',
 'accopenpast24mths',
 'acceptd',
 'addrstate',
 'addrzip',
 'allutil',
 'annualinc',
 'annualincjoint',
 'applicationtype',
 'avgcurbal',
 'bcopentobuy',
 'bcutil',
 'chargeoffwithin12mths',
 'collections12mthsexmed',
 'creditpulld',
 'delinq2yrs',
 'delinqamnt',
 'desc',
 'disbursementmethod',
 'dti',
 'dtijoint',
 'earliestcrline',
 'emplength',
 'emptitle',
 'expd',
 'expdefaultrate',
 'ficorangehigh',
 'ficorangelow',
 'fundedamount',
 'grade',
 'homeownership',
 'housingpayment',
 'ilutil',
 'id',
 'ilsexpd',
 'initialliststatus',
 'inqfi',
 'inqlast12m',
 'inqlast6mths',
 'installment',
 'intrate',
 'investorcount',
 'isincv',
 'isincvjoint',
 'listd',
 'loanamount',
 'maxbalbc',
 'memberid',
 'mosinoldilacct',
 'mosinoldrevtlop',
 'mosinrcntrevtlop',
 'mosinrcnttl',
 'mortacc',
 'mtgpayment',
 'mthssincelastdelinq',
 'mthssincelastmajorderog',
 'mthssincelastrecord',
 'mthssincercntil',
 'mthssincerecentbc',
 'mthssincerecentbcdlq',
 'mthssincerecentinq',
 'mth

In [22]:
# save df_2014 feature into his_feature
his_feature = list(df_2014.columns.values)

In [23]:
his_feature

['numtl120dpd2m',
 'totalbalexmort',
 'inqfi',
 'accopenpast24mths',
 'avgcurbal',
 'id',
 'mosinoldrevtlop',
 'openacc',
 'numrevtlbalgt0',
 'inqlast6mths',
 'totalilhighcreditlimit',
 'revolutil',
 'grade',
 'mortacc',
 'mthssincercntil',
 'mthssincelastrecord',
 'tothicredlim',
 'memberid',
 'mthssincerecentbc',
 'numtl90gdpd24m',
 'maxbalbc',
 'numiltl',
 'totalrevhilim',
 'mthssincerecentbcdlq',
 'ficorangelow',
 'mthssincerecentrevoldelinq',
 'initialliststatus',
 'numbcsats',
 'numsats',
 'bcopentobuy',
 'openil24m',
 'applicationtype',
 'delinqamnt',
 'openrv24m',
 'percentbcgt75',
 'taxliens',
 'installment',
 'ficorangehigh',
 'mosinoldilacct',
 'subgrade',
 'numactvrevtl',
 'numtloppast12m',
 'openrv12m',
 'numactvbctl',
 'chargeoffwithin12mths',
 'numbctl',
 'desc',
 'allutil',
 'ilutil',
 'pubrec',
 'bcutil',
 'pcttlnvrdlq',
 'totalcutl',
 'mthssincelastmajorderog',
 'accnowdelinq',
 'totalbalil',
 'totcollamt',
 'mthssincerecentinq',
 'mosinrcntrevtlop',
 'openil12m',
 't

## find common_feature existing in both current data and 2014 data

In [24]:
common_feature = list(set(his_feature)&(set(current_feature)))
print ("number of common features: " + str(len(common_feature)) + '\n')
print ('----------common features-----------------')
common_feature

number of common features: 86

----------common features-----------------


['homeownership',
 'term',
 'percentbcgt75',
 'totalcutl',
 'mthssincerecentbc',
 'revolutil',
 'numiltl',
 'emplength',
 'totcollamt',
 'maxbalbc',
 'annualinc',
 'inqlast6mths',
 'totalilhighcreditlimit',
 'numtl120dpd2m',
 'numrevtlbalgt0',
 'pubrecbankruptcies',
 'mthssincelastdelinq',
 'taxliens',
 'mosinoldrevtlop',
 'openacc',
 'openrv24m',
 'totalbclimit',
 'grade',
 'mthssincelastmajorderog',
 'totalacc',
 'numbctl',
 'delinqamnt',
 'bcutil',
 'numoprevtl',
 'ficorangehigh',
 'accopenpast24mths',
 'mthssincerecentinq',
 'initialliststatus',
 'totalbalexmort',
 'applicationtype',
 'subgrade',
 'mthssincelastrecord',
 'ilutil',
 'mthssincerecentrevoldelinq',
 'mosinrcnttl',
 'numbcsats',
 'earliestcrline',
 'desc',
 'chargeoffwithin12mths',
 'emptitle',
 'tothicredlim',
 'openrv12m',
 'accnowdelinq',
 'mthssincerecentbcdlq',
 'avgcurbal',
 'openil12m',
 'id',
 'delinq2yrs',
 'intrate',
 'mthssincercntil',
 'numrevaccts',
 'bcopentobuy',
 'dti',
 'installment',
 'annualincjoint',

find historical features only existing in 2014 data  --> his_unique 

In [25]:
his_unique= list(set(his_feature)-set(current_feature))

print ("number of features only exist in 2014 data: " + str(len(his_unique)))
print ('\n')
print ('----------2014_only features-----------------')
his_unique

number of features only exist in 2014 data: 8


----------2014_only features-----------------


['issued',
 'verificationstatusjoint',
 'verificationstatus',
 'loanamnt',
 'zipcode',
 'fundedamnt',
 'numacctsever120pd',
 'loanstatus']


find current features only existing in current data --> current_unique

In [26]:
current_unique=list(set(current_feature)-set(his_feature))
print ("number of features only exist in current data: " + str(len(current_unique)))
print ('\n')
print ('----------current_only features-----------------')
current_unique

number of features only exist in current data: 33


----------current_only features-----------------


['housingpayment',
 'acceptd',
 'secappmortacc',
 'numacctsever120ppd',
 'expd',
 'reviewstatusd',
 'secappcollections12mthsexmed',
 'addrzip',
 'secappnumrevaccts',
 'secappficorangelow',
 'isincvjoint',
 'secappopenactil',
 'mtgpayment',
 'secappinqlast6mths',
 'secappchargeoffwithin12mths',
 'secapprevolutil',
 'loanamount',
 'reviewstatus',
 'secappficorangehigh',
 'secappearliestcrline',
 'disbursementmethod',
 'isincv',
 'secappopenacc',
 'servicefeerate',
 'investorcount',
 'listd',
 'expdefaultrate',
 'ilsexpd',
 'secappmthssincelastmajorderog',
 'creditpulld',
 'revolbaljoint',
 'fundedamount',
 'openactil']

## save common_feature, his_unique and current feature

note that several features have different names in 2014 and current data, so it is necessary to manually  add them into the common feature 

In [27]:
# mannually add several common features that are shared by current and historical data but have different names
common_feature=common_feature + ['zipcode', 
 'fundedamnt',
 'verificationstatus',
 'verificationstatusjoint',
 'loanamnt',
 'numacctsever120pd']

len(common_feature)

92

In [28]:
# use dict feature_dic to save common_feature, his_unique feature & current_unique 
feature_dic={}
feature_dic['common_feature']=common_feature
feature_dic['his_unique']=his_unique
feature_dic['current_unique']=current_unique

# 4. Data Wrangling

In [29]:
df_2014.shape

(235631, 94)

In [30]:
df_2014.head()

,numtl120dpd2m,totalbalexmort,inqfi,accopenpast24mths,avgcurbal,id,mosinoldrevtlop,openacc,numrevtlbalgt0,inqlast6mths,...,intrate,addrstate,zipcode,fundedamnt,verificationstatus,verificationstatusjoint,loanamnt,numacctsever120pd,issued,loanstatus
0,0.0,15030.0,NaN,7.0,9536.0,36805548,290.0,17.0,9.0,2.0,...,6.99%,CA,937xx,10400.0,Not Verified,NaN,10400.0,4.0,Dec-2014,Charged Off
1,0.0,149140.0,NaN,5.0,29828.0,38098114,244.0,6.0,4.0,0.0,...,12.39%,VA,235xx,15000.0,Source Verified,NaN,15000.0,0.0,Dec-2014,Fully Paid
2,0.0,38566.0,NaN,8.0,3214.0,37822187,265.0,12.0,7.0,0.0,...,13.66%,NJ,077xx,9600.0,Source Verified,NaN,9600.0,0.0,Dec-2014,Fully Paid
3,0.0,64426.0,NaN,6.0,5857.0,37662224,148.0,11.0,4.0,1.0,...,13.66%,AZ,850xx,7650.0,Source Verified,NaN,7650.0,0.0,Dec-2014,Charged Off
4,0.0,18007.0,NaN,2.0,32727.0,37612354,170.0,8.0,5.0,0.0,...,17.14%,CA,953xx,12800.0,Verified,NaN,12800.0,1.0,Dec-2014,Current


In [31]:
df_2014.tail()

,numtl120dpd2m,totalbalexmort,inqfi,accopenpast24mths,avgcurbal,id,mosinoldrevtlop,openacc,numrevtlbalgt0,inqlast6mths,...,intrate,addrstate,zipcode,fundedamnt,verificationstatus,verificationstatusjoint,loanamnt,numacctsever120pd,issued,loanstatus
235626,0.0,50896.0,NaN,7.0,28172.0,9604874,130.0,21.0,3.0,1.0,...,7.90%,CA,913xx,2000.0,Verified,NaN,2000.0,0.0,Jan-2014,Fully Paid
235627,0.0,73598.0,NaN,8.0,4089.0,9584776,93.0,18.0,2.0,2.0,...,16.99%,OH,458xx,20700.0,Verified,NaN,20700.0,0.0,Jan-2014,Current
235628,0.0,57477.0,NaN,4.0,11495.0,9199665,166.0,6.0,2.0,0.0,...,19.20%,CA,950xx,10000.0,Verified,NaN,10000.0,0.0,Jan-2014,Fully Paid
235629,NaN,NaN,NaN,NaN,NaN,Total amount funded in policy code 1: 3503840175,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
235630,NaN,NaN,NaN,NaN,NaN,Total amount funded in policy code 2: 873652739,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


need to remove the last two rows because of invalid data

In [32]:
# remove last two rows
df_2014 = df_2014.iloc[:-2]
df_2014.shape

(235629, 94)

remove featues that are not existing in 2014 data

Only keep common features

In [33]:
df_2014.term.value_counts()

 36 months    162570
 60 months     73059
Name: term, dtype: int64

only use term=36 months 

In [34]:
df_2014 = df_2014.query("term ==' 36 months'")

In [35]:
df_2014.shape

(162570, 94)

only use loanstatus = fully paid and charged off

In [36]:
df_2014.loanstatus.value_counts()

Fully Paid            108778
Current                31114
Charged Off            20709
Late (31-120 days)      1045
In Grace Period          726
Late (16-30 days)        194
Default                    4
Name: loanstatus, dtype: int64

In [37]:
df_2014 = df_2014.query("loanstatus == 'Fully Paid' or loanstatus == 'Charged Off' ")

In [38]:
df_2014.loanstatus.value_counts()

Fully Paid     108778
Charged Off     20709
Name: loanstatus, dtype: int64

In [39]:
df_2014.shape

(129487, 94)

removal all null featues

In [40]:
df_2014.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 129487 entries, 0 to 235628
Data columns (total 94 columns):
numtl120dpd2m                 123910 non-null float64
totalbalexmort                129487 non-null float64
inqfi                         0 non-null float64
accopenpast24mths             129487 non-null float64
avgcurbal                     129482 non-null float64
id                            129487 non-null object
mosinoldrevtlop               129487 non-null float64
openacc                       129487 non-null float64
numrevtlbalgt0                129487 non-null float64
inqlast6mths                  129487 non-null float64
totalilhighcreditlimit        129487 non-null float64
revolutil                     129428 non-null object
grade                         129487 non-null object
mortacc                       129487 non-null float64
mthssincercntil               0 non-null float64
mthssincelastrecord           25307 non-null float64
tothicredlim                  129487 no

In [41]:
df_2014['loanstatus'].isnull().sum()

0

In [42]:
all_null_feature = []
ob_feature = []
num_feature = []
for col in df_2014.columns.values:
    if df_2014[col].isnull().sum() == df_2014.shape[0]:
        all_null_feature.append(col)
    elif df_2014[col].dtypes == 'object':
        ob_feature.append(col)
    else:
        num_feature.append(col)
    
        

In [43]:
print ("all_null_feature have " + str(len(all_null_feature)) + " features")
print ("ob_feature have " + str(len(ob_feature)) + " features")
print ("num_feature have " + str(len(num_feature)) + " features")

all_null_feature have 17 features
ob_feature have 19 features
num_feature have 58 features


In [44]:
select_feature=ob_feature + num_feature
len(select_feature)

77

remove low information features

In [45]:
#application type only has 'individual"
ob_feature.remove('applicationtype')

#no need for id, loanstatus and issued
# ob_feature.remove('id')
# ob_feature.remove('loanstatus')
# ob_feature.remove('issued')

remove open form feature

In [46]:
# remove 'desc' from ob_feature and select_feature
ob_feature.remove('desc')

remove redudant features

In [47]:
# remove fundedamnt since it always equals to 1 for issued loans and varied during different time of a current loan

num_feature.remove('fundedamnt')

In [48]:
select_feature=ob_feature+num_feature
len(select_feature)

74

In [49]:
df_2014=df_2014[select_feature]

In [50]:
df_2014.shape

(129487, 74)

In [51]:
df_2014.verificationstatus.value_counts()

Not Verified       47507
Source Verified    47370
Verified           34610
Name: verificationstatus, dtype: int64

In [52]:
df_2014.loanstatus.value_counts()

Fully Paid     108778
Charged Off     20709
Name: loanstatus, dtype: int64

In [53]:
df_2014['loanstatus'] = df_2014.loanstatus.map({"Charged Off": 1, "Fully Paid": 0})

In [54]:
df_2014.loanstatus.value_counts()

0    108778
1     20709
Name: loanstatus, dtype: int64

 ### Datetime to numeric feature

earliestcrline: The month the borrower's earliest reported credit line was opened

In [55]:
df_2014.earliestcrline.unique()[:5]

array(['Sep-1989', 'Nov-1992', 'Aug-2002', 'Oct-1998', 'Jan-2001'], dtype=object)

convert to number of months to Dec 2014

In [56]:
cl_month=df_2014.earliestcrline.apply(lambda x: x.split('-')[0])
cl_year=df_2014.earliestcrline.apply(lambda x: int(x.split('-')[1]))

In [57]:
cl_month.unique()

array(['Sep', 'Nov', 'Aug', 'Oct', 'Jan', 'May', 'Jul', 'Dec', 'Jun',
       'Mar', 'Apr', 'Feb'], dtype=object)

In [58]:
cl_year.unique()

array([1989, 1992, 2002, 1998, 2001, 1999, 2009, 2003, 1993, 1995, 1996,
       2007, 2004, 1997, 2008, 2011, 2006, 2010, 1988, 1994, 2000, 1987,
       2005, 1986, 1973, 1982, 1991, 1984, 1976, 1974, 1981, 1990, 1978,
       1979, 1985, 1983, 1971, 1967, 1972, 1968, 1980, 1977, 1965, 1966,
       1975, 1969, 1963, 1970, 1964, 1958, 1962, 1959, 1955, 1960, 1961,
       1954, 1957, 1949, 1950, 1948, 1953, 1956])

In [59]:
dic_month= {'Jan':11,'Feb':10,'Mar':9,'Apr':8, 'May':7, 'Jun':6, 'Jul':5, 'Aug':4, 'Sep':3, 'Oct':2, 'Nov':1, 'Dec':0}

In [60]:
df_2014['earliestcrline_month']=df_2014.earliestcrline.apply(lambda x: (2014-int(x.split('-')[1]))*12+dic_month[x.split('-')[0]])

In [61]:
df_2014.earliestcrline_month[:5]

0    303
2    265
3    148
6    194
7    167
Name: earliestcrline_month, dtype: int64

In [62]:
num_feature.append('earliestcrline_month')
ob_feature.remove('earliestcrline')

emplength

In [63]:
#check number of null values
print (df_2014['emplength'].isnull().sum())
df_2014['emplength'].unique()

7321


array(['8 years', '10+ years', '< 1 year', '2 years', '7 years', '9 years',
       nan, '4 years', '3 years', '1 year', '5 years', '6 years'], dtype=object)

In [64]:
df_2014['emplength'].replace('n/a', np.nan, inplace=True)
df_2014['emplength'].replace('< 1 year', '0', inplace=True)
df_2014['emplength'].replace(to_replace='[^0-9]+', value='', inplace=True, regex=True)
df_2014.emplength.fillna(value=-999,inplace=True)
df_2014['emplength'] = df_2014['emplength'].astype(int)

In [65]:
df_2014.emplength.value_counts()

 10     40456
 2      11807
 0      10538
 3      10523
 1       8502
 4       7572
 5       7395
-999     7321
 7       7211
 6       6704
 8       6457
 9       5001
Name: emplength, dtype: int64

In [66]:
num_feature.append('emplength')
ob_feature.remove('emplength')
len(num_feature), len(ob_feature)

(59, 15)

In [67]:
## % to numeric feature: intrate, revolutil

In [68]:
intrate=df_2014.intrate.apply(lambda x: float(x.replace('%','')))

In [69]:
df_2014.intrate=intrate
df_2014.intrate[:5]

0     6.99
2    13.66
3    13.66
6    14.31
7    13.66
Name: intrate, dtype: float64

In [70]:
revolutil=df_2014.revolutil.apply(lambda x: str(x).replace('%','')).astype(float)

df_2014.revolutil=revolutil
df_2014.revolutil[:5]

0    31.6
2    59.4
3    91.9
6    82.2
7    46.4
Name: revolutil, dtype: float64

In [71]:
ob_feature.remove('intrate')
num_feature.append('intrate')
ob_feature.remove('revolutil')
num_feature.append('revolutil')

In [72]:
len(ob_feature), len(num_feature)

(13, 61)

Ordinal feature encoding : grade, subgrade

In [73]:
Dic_grade = {"A": 1, 
            "B": 2, 
            "C": 3, 
            "D": 4, 
            "E": 5,
            "F": 6, 
            "G": 7}

In [74]:
df_2014.grade = df_2014.grade.map(Dic_grade)

In [75]:
df_2014.subgrade = df_2014.subgrade.apply(lambda x: (Dic_grade[x[0]] - 1) * 5 + int(x[1]))

In [76]:
df_2014.subgrade.unique()

array([ 3, 13, 14, 10, 20,  9, 17,  8, 15, 12,  7, 11, 25, 18,  6,  5,  4,
       16, 21,  1,  2, 22, 27, 19, 29, 23, 24, 26, 28, 31, 32, 30, 33, 34,
       35])

In [77]:
ob_feature.remove('grade')
num_feature.append('grade')
ob_feature.remove('subgrade')
num_feature.append('subgrade')

In [78]:
len(ob_feature), len(num_feature)

(11, 63)

In [79]:
### High cardinality feature encoding

In [80]:
###### Zip Code - frequency encoding 

In [81]:
df_2014.zipcode.nunique()

850

In [82]:
df_2014.zipcode.unique()[:5]

array(['937xx', '077xx', '850xx', '982xx', '144xx'], dtype=object)

In [83]:
# convert zipcode to int
df_2014.zipcode = df_2014.zipcode.apply(lambda x: int(x[0:3]))

In [84]:
df_2014.zipcode[:5]

0    937
2     77
3    850
6    982
7    144
Name: zipcode, dtype: int64

In [85]:
zipcode_freq = df_2014.groupby("zipcode").size().reset_index()

In [86]:
zipcode_freq.columns = ["zipcode", "zipcode_freq"]
zipcode_freq[0:5]

,zipcode,zipcode_freq
0,8,1
1,10,180
2,11,46
3,12,47
4,13,32


In [87]:
df_2014 = pd.merge(df_2014, zipcode_freq, how = "left", on = "zipcode")

In [88]:
df_2014.zipcode_freq[:5]

0    203
1    288
2    602
3    381
4    163
Name: zipcode_freq, dtype: int64

In [89]:
ob_feature.remove('zipcode')
num_feature.append('zipcode')
num_feature.append('zipcode_freq')
len(ob_feature), len(num_feature)

(10, 65)

In [90]:
### emptitle-frequency encoding (can do some NLP for later stage)

In [91]:
emptitle_freq = df_2014.groupby("emptitle").size().reset_index()
emptitle_freq.columns = ["emptitle", "emptitle_freq"]
df_2014 = pd.merge(df_2014, emptitle_freq, how = "left", on = "emptitle")


In [92]:
df_2014.emptitle.head()

0    Truck Driver Delivery Personel
1                  Admin Specialist
2              Technical Specialist
3                           Teacher
4                    Deputy sheriff
Name: emptitle, dtype: object

In [93]:
ob_feature.remove('emptitle')
num_feature.append('emptitle_freq')
len(ob_feature), len(num_feature)

(9, 66)

In [94]:
###### Addr_state - frequency encoding

In [95]:
addrstate_freq = df_2014.groupby("addrstate").size().reset_index()
addrstate_freq.columns = ["addrstate", "addrstate_freq"]
df_2014 = pd.merge(df_2014, addrstate_freq, how = "left", on = "addrstate")


In [96]:
ob_feature.remove('addrstate')
num_feature.append('addrstate_freq')
len(ob_feature), len(num_feature)

(8, 67)

remove term as it only contains 36 month loans

In [97]:
ob_feature.remove('term')

In [98]:
len(ob_feature), len(num_feature)

(7, 67)

# 4. Save processed data in loan_2014_processed.csv

In [102]:
select_feature = ob_feature + num_feature

In [103]:
len(select_feature)

74

In [104]:
df_2014 = df_2014[select_feature]

it is also necessary to convert column names in the dataframe directly

In [105]:
# convert column name in df_2014
for col in df_2014.columns.values:
    df_2014.rename(columns={str(col):str(col).replace('_','')}, inplace=True)


In [106]:
# save dataframe into loan_2014.csv
df_2014.to_csv('data/loan_2014_processed.csv',index=False)

In [107]:
# check saved csv file
df=pd.read_csv('data/loan_2014_processed.csv')

In [108]:
df.shape

(129487, 74)

In [109]:
df.columns.values

array(['id', 'initialliststatus', 'homeownership', 'purpose',
       'verificationstatus', 'issued', 'loanstatus', 'numtl120dpd2m',
       'totalbalexmort', 'accopenpast24mths', 'avgcurbal',
       'mosinoldrevtlop', 'openacc', 'numrevtlbalgt0', 'inqlast6mths',
       'totalilhighcreditlimit', 'mortacc', 'mthssincelastrecord',
       'tothicredlim', 'mthssincerecentbc', 'numtl90gdpd24m', 'numiltl',
       'totalrevhilim', 'mthssincerecentbcdlq', 'ficorangelow',
       'mthssincerecentrevoldelinq', 'numbcsats', 'numsats', 'bcopentobuy',
       'delinqamnt', 'percentbcgt75', 'taxliens', 'installment',
       'ficorangehigh', 'mosinoldilacct', 'numactvrevtl', 'numtloppast12m',
       'numactvbctl', 'chargeoffwithin12mths', 'numbctl', 'pubrec',
       'bcutil', 'pcttlnvrdlq', 'mthssincelastmajorderog', 'accnowdelinq',
       'totcollamt', 'mthssincerecentinq', 'mosinrcntrevtlop',
       'totalbclimit', 'pubrecbankruptcies', 'numoprevtl', 'revolbal',
       'annualinc', 'totalacc', 'mthssin